In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, MaxPooling2D, Conv2D, Flatten, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop

In [2]:
import glob as gb 
import cv2

In [3]:
IMG_SIZE = 224

TRAINING_DIR = '../data/familia_simpsons_train/'

training_datagen = ImageDataGenerator(rescale=1./255, featurewise_center=False, 
 samplewise_center=False, 
 featurewise_std_normalization=False,  
 samplewise_std_normalization=False, 
 rotation_range=0, 
 width_shift_range=0.1,  
 height_shift_range=0.1,  
 horizontal_flip=True, 
 vertical_flip=False)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR,target_size=(IMG_SIZE,IMG_SIZE), class_mode='categorical', batch_size=64, shuffle=True)

Found 7274 images belonging to 6 classes.


In [4]:
class_map = train_generator.class_indices
inv_class_map = {v: k for k, v in class_map.items()}
inv_class_map

{0: 'abraham_grampa_simpson',
 1: 'bart_simpson',
 2: 'homer_simpson',
 3: 'lisa_simpson',
 4: 'maggie_simpson',
 5: 'marge_simpson'}

In [5]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(IMG_SIZE, IMG_SIZE, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=1024, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
                                  
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(6, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [6]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

EPOCHS = 5

history = model.fit(train_generator, batch_size=64, epochs=EPOCHS)

Epoch 1/5
114/114 [==============================] - 663s 6s/step - loss: 3.2127 - accuracy: 0.2947
Epoch 2/5
114/114 [==============================] - 645s 6s/step - loss: 1.3449 - accuracy: 0.4488
Epoch 3/5
114/114 [==============================] - 651s 6s/step - loss: 1.0314 - accuracy: 0.5925
Epoch 4/5
114/114 [==============================] - 650s 6s/step - loss: 0.8009 - accuracy: 0.6963
Epoch 5/5
114/114 [==============================] - 633s 6s/step - loss: 0.5837 - accuracy: 0.7808


In [7]:
#model.save('familia_simpsons_model.h5')

In [13]:
testpath = '../data/familia_simpsons_test/'
acum = 0
for files in  os.listdir(testpath):
    acum += 1
print(f'For testing data , found {acum} files')

For testing data , found 248 files


In [18]:
s = 224

In [19]:
X_test = []
files = gb.glob(str(testpath + '/*.jpg'))
for file in files: 
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_array = cv2.resize(image , (s,s))
    X_test.append(list(image_array))

In [20]:
X_test = np.array(X_test, dtype = 'float32')
print(f'X_test shape : {X_test.shape}')
X_test = X_test/255.0

X_test shape : (248, 224, 224, 3)


In [21]:
predictions = model.predict(X_test)
predictions_1 = np.argmax(predictions, axis=-1)
predictions_1

array([0, 2, 0, 2, 0, 2, 2, 2, 1, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 3, 1, 0, 1, 2, 1, 1, 3, 1, 1, 1, 1,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5], dtype=int64)